#### ```0. Basic_check(df)```
- pd.sample 확인, info, 전체 데이터 수 및 null 값 현황을 확인하는 함수  
---

### 1. 날짜 관련 함수 처리
#### ```1-1. date_preprocess(df)```
- tran = tran_date + tran_time
- play = play_date + play_st_time
- 시간 데이터 형식을 4자리 수로 변환 ex) 615->0615
- 날짜와 시간을 합쳐서 datetime 형식 데이터로 변경함
- 각 날짜별 요일을 한국어로 추출하여 tran_day, play_day col로 추가
- play_day == 월요일인 데이터 삭제
- float 타입의 데이터(pre_open_date, open_date)들을 datetime(yyyy-mm-dd 형식)으로 변경
---
#### ```1-2. tran_rank_min(df)```
- 'performance_code'로 그룹화하여 'tran' 열을 가장 빠른 순서대로 정렬 후, rank를 매김
- 최종적으로 사용한 method = 'min' 이지만, method = 'first'와 비교하여 확인한 과정도 남겨두었음. 
- 특정 공연에서 빨리 예매된 좌석이 어디인지를 파악하고 그 좌석이 좋은 좌석이라는 feature로 사용하기 위한 작업
---
### ```2. 데이터 나누기```
### ```2-1. sep_seat_(df)```
- 좌석 정보 분리하여 4개의 col에 저장 : seat_floor(층),seat_block(블록),seat_col(열),seat_num(좌석번호)
### ```2-2. sep_discount(df)```
- 할인명, 할인률 구분하기   
---
### `3. 오류 데이터 처리`
- member_yn = 'N'인 데이터 중에서 멤버십, 젠더 정보가 있는 경우 'Y'로 변경`
---
### `4. 데이터 이진화`
- pre_open_date, gender, member_yn 데이터 2진화 수행, gender의 경우 nan을 2로 labeling  
** pre_open_date : 예술의전당 유료회원 상대로 미리 예매를 할 수 있는 하루 전 혜택, 기획자/대관자의 선택에 의해 결정  

---
### `5. membership column 정리`
- 6개의 membership_type 컬럼을 membership_free, membership_paid 2개의 컬럼으로 정리  
- free : 노블, 싹틔우미, 일반  
- paid : 법인, 골드, 블루, 그린, 무료로 등급을 나누어 컬럼 간소화  

---
### `6. age 결측값 대치`  
- age가 nan, membership_free 존재 => free 등급 기준 평균대치 (일반의 경우 50대로 대치)  
- age가 nan, membership_free 미존재 => 삭제  
  
### `7. genre nan 값 처리`
- genre : nan값에 '기타' 부여

In [1]:
# !pip install -r requirements.txt
# !pip install pandas matplotlib seaborn

In [2]:
import pandas as pd 
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings

plt.rc('font', family = 'Malgun Gothic')
filterwarnings('ignore')

# 전체 데이터 출력 및 확인 옵션 설정
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_info_columns

100

# 0. basic_check
샘플 데이터 print, info, 전체 데이터 수 및 null 값 현황을 확인하는 함수

In [3]:
def basic_check(df):
    # 샘플 데이터 10개 확인
    print('샘플 데이터 10개 확인\n', df.sample(10))
    print('==================================================')
    # info 확인
    print('info 확인\n', df.info())
    print('==================================================')
    # 전체 데이터 수 및 null 값 확인
    print('전체 데이터 수 : ', len(df)) 
    print('null 값 확인\n', df.isnull().sum())

# 1-1. date_preprocess
* input
  - df : 데이터프레임
* output
  - df : 데이터프레임
  - tran : tran_date + tran_time, datetime 형태로 가공(yyyy-mm-dd hh:mm:ss)
  - play : play_date + play_st_time, datetime 형태로 가공(yyyy-mm-dd hh:mm:ss)
  - tran_day : tran에서 한국어 요일 정보 추출
  - play_day : play에서 한국어 요일 정보 추출 후, '월요일' 삭제
  - float 타입의 데이터(pre_open_date, open_date)들을 datetime(yyyy-mm-dd 형식)으로 변경

In [12]:
def date_preprocess(df):
    df_new = df.copy()
    # 1. tran 처리
    # 시간, 분을 4자리 수로 변환 ex) 615->0615
    df_new['tran_time'] = df_new['tran_time'].astype(str).str.zfill(4)
    df_new['tran'] = df_new['tran_date'].astype(str) + df_new['tran_time']
    df_new['tran'] = pd.to_datetime(df_new['tran'], format='%Y%m%d%H%M') # datetime화
    # 요일 생성
    df_new['tran_day'] = df_new['tran'].dt.day_name(locale='ko_kr') #locale 미설정시 영어로 나옴

    # 2. play 처리
    # 시간, 분을 4자리 수로 변환 ex) 615->0615
    df_new['play_st_time'] = df_new['play_st_time'].astype(str).str.zfill(4)
    df_new['play'] = df_new['play_date'].astype(str) + df_new['tran_time']
    df_new['play'] = pd.to_datetime(df_new['play'], format='%Y%m%d%H%M') # datetime화
    # 요일 생성
    df_new['play_day'] = df_new['play'].dt.day_name(locale='ko_kr') #locale 미설정시 영어로 나옴
    df_new = df_new[df_new['play_day'] != '월요일']

    #3. 기존 열들은 삭제
    df_new = df_new.drop(['tran_date', 'tran_time', 'play_date', 'play_st_time'], axis=1)

    #4.'pre_open_date', 'open_date' datetime화
    df_new['pre_open_date'] = pd.to_datetime(df_new['pre_open_date'], format='%Y%m%d')
    df_new['open_date'] = pd.to_datetime(df_new['open_date'], format='%Y%m%d')
    return df_new

# 1-2. tran_rank_dense(df)
- 'performance_code'로 그룹화하여 'tran' 열을 가장 빠른 순서대로 정렬 후, rank를 매김
- 최종적으로 사용한 method = 'min' 이지만, method = 'first'와 비교하여 확인한 과정도 남겨두었음.  
---
- 특정 한 공연에서 순위를 확인했을 때, method = 'first'를 사용하면 먼저 나타난 데이터에 우선순위를 부여함.
- 그러나 tran이 같은(즉, 동시에 예매한 경우) 경우가 존재함.
- 이 과정은 특정 공연에서 빨리 예매된 좌석이 어디인지를 파악하고 그 좌석이 좋은 좌석이라는 feature로 사용하고자 하므로
- method = 'dense'을 사용해서 1,2,3이 아니라 1,1,2과 같이 순위를 매기기로 하였음.

In [12]:
# rank method = dense
def tran_rank_dense(df):    
    df['tran_rank'] = df.groupby('performance_code')['tran'].rank(ascending=True, method='dense')
    return df

# 2-1. sep_seat(df)
- 좌석 정보 분리하여 4개의 col에 저장 : seat_floor(층),seat_block(블록),seat_col(열),seat_num(좌석번호)   

In [15]:
def sep_seat(df):
    # 0. place별로 df 구분
    df_concert = df.query("`place`=='콘서트홀'")
    df_chamber = df.query("`place`=='IBK챔버홀'")
    df_recital = df.query("`place`=='리사이틀홀'")


    # ===concert 홀 처리===
    # 1.seat_floor 생성
    df_concert['seat_floor'] = df_concert['seat'].str.split(" ").str[0]
    df_concert['seat_floor'] = df_concert['seat_floor'].replace("층$","",regex=True) # 마지막 글자가 '층'이면 삭제
    df_concert['seat_floor'] = df_concert['seat_floor'].replace("합창석",4,regex=True) # 3층까지만 있으므로, 합창석은 4로 대체

    # 2. seat_block, seat_col 생성 
    # 2-1. BOX1과 같이 "블록"으로 split 불가한 경우 대비 : seat_block에 넣어두고, 이 경우 seat_col은 nan이 되므로 -1 부여
    df_concert['seat_block'] = df_concert['seat'].str.split(" ").str[1]
    df_concert['seat_col'] = -1

    # 2-2. box df, block df 두 가지로 구분
    df_concert_box = df_concert.query('not seat_block.str.contains("블록")')
    df_concert_block = df_concert.query('seat_block.str.contains("블록")')

    # 2-3. block df에서 "블록"기준으로 구분하여 담고, "열"글자 삭제
    df_concert_block[['seat_block', 'seat_col']] = df_concert_block['seat_block'].str.split("블록", expand=True)
    df_concert_block['seat_col'] = df_concert_block['seat_col'].replace("열$","",regex=True) # 마지막 글자가 '열'이면 삭제

    df_concert_new = pd.concat([df_concert_box, df_concert_block])

    # 3. seat_num 생성
    df_concert_new['seat_num'] = df_concert_new['seat'].str.split(" ").str[-1]


    # ===chamber 홀 처리===
    # 1.seat_floor 생성
    df_chamber['seat_floor'] = df_chamber['seat'].str.split(" ").str[0]
    df_chamber['seat_floor'] = df_chamber['seat_floor'].replace("층$","",regex=True) # 마지막 글자가 '층'이면 삭제. 1층과 2층만 있음

    # 2. seat_block 생성 
    df_chamber['seat_block'] = df_chamber['seat'].str.split(" ").str[1]

    # 3. seat_col 생성 :  box df, block df 두 가지로 구분
    df_chamber_box = df_chamber.query('not seat_block.str.contains("블록")')
    # 3-1. BOX석이 seat_block으로 입력될 때 seat_col은 nan이 되므로 -1 부여
    df_chamber_box['seat_col'] = -1

    # 3-2. 블록이 있으면 구분되어 있으므로 데이터 부여
    df_chamber_block = df_chamber.query('seat_block.str.contains("블록")')

    # 3-3. seat 데이터를 이용해 공백 기준으로 구분하여 담고, "블록"&"열"글자 삭제
    df_chamber_block['seat_col'] = df_chamber_block['seat'].str.split(" ").str[2]
    df_chamber_block['seat_block'] = df_chamber_block['seat_block'].replace("블록$","",regex=True) # 마지막 글자가 '블록'이면 삭제
    df_chamber_block['seat_col'] = df_chamber_block['seat_col'].replace("열$","",regex=True) # 마지막 글자가 '열'이면 삭제

    df_chamber_new = pd.concat([df_chamber_box, df_chamber_block])

    # 4. seat_num 생성
    df_chamber_new['seat_num'] = df_chamber_new['seat'].str.split(" ").str[-1]

    # ===recital 홀 처리===
    # 1.seat_floor 생성
    df_recital['seat_floor'] = df_recital['seat'].str.split(" ").str[0]
    df_recital['seat_floor'] = df_recital['seat_floor'].replace("층$","",regex=True) # 마지막 글자가 '층'이면 삭제. 1층과 2층만 있음

    # 2. seat_block 생성 : block, noblock으로 구분 : split list의 길이가 3이면 nan 값이므로 -1 부여
    df_recital_noblock = df_recital[df_recital['seat'].str.split(" ").str.len() < 4]
    df_recital_noblock['seat_block'] = -1

    df_recital_block = df_recital[df_recital['seat'].str.split(" ").str.len() == 4]
    df_recital_block['seat_block'] = df_recital_block['seat'].str.split(" ").str[1]

    df_recital_new = pd.concat([df_recital_noblock, df_recital_block])

    # 3. seat_col 생성 : 뒤에서 두 번째 값으로 부여
    df_recital_new['seat_col'] = df_recital_new['seat'].str.split(" ").str[-2]
    df_recital_new['seat_col'] = df_recital_new['seat_col'].replace("열$","",regex=True) # 마지막 글자가 '열'이면 삭제

    # 3. seat_num 생성
    df_recital_new['seat_num'] = df_recital_new['seat'].str.split(" ").str[-1]

    
    df_new = pd.concat([df_concert_new, df_chamber_new, df_recital_new])
    df_new[['seat_floor', 'seat_col', 'seat_num']] = df_new[['seat_floor', 'seat_col', 'seat_num']].astype(int)
    result = df_new.drop(['seat'], axis=1)
    return result.sort_index()

# 2-2. sep_discount()
- 1. %가 포함된 것과, 아닌 것으로 분류  
- 2. %가 포함되어 있지 않은 df_plain 처리  
  if price == 0 : 100 (초대권인데 돈을 낸 경우도 존재)  
  if price != 0 : '일반'이면 0, 그 외에는 할인율을 알 수 없기 때문에 -1로 처리  
- 3. %가 포함된 df_rate 처리  
  if price == 0 : df_rate에서도 가격이 0이라면 discount_rate는 100이므로, % 데이터를 무시하고 100으로 처리.  
  if price != 0 : "~%할인", "~%_"로 끝나는 데이터 처리 후, 할인 이름과 할인률을 추출.  
- 4. df_new 생성, data type 통일  

In [14]:
def sep_discount(df):
    # 1. %가 포함된 것과, 아닌 것으로 분류
    df_plain = df.query('not discount_type.str.contains("%")')
    df_rate = df.query('discount_type.str.contains("%")')
    
    # 2. %가 포함되어 있지 않은 df_plain 처리
    # if price == 0 : 100 (초대권인데 돈을 낸 경우도 존재)
    # if price != 0 : '일반'이면 0, 그 외에는 할인율을 알 수 없기 때문에 -1로 처리
    df_plain100 = df_plain[df_plain['price'] == 0]
    df_plain100['discount_name'] = df_plain100['discount_type']
    df_plain100['discount_rate'] = 100

    df_plain100x = df_plain[df_plain['price'] != 0]
    df_plain100x = df_plain100x[df_plain100x['discount_type'] == "일반"]
    df_plain100x['discount_name'] = "일반"
    df_plain100x['discount_rate'] = 0

    df_etc = df_plain100x[df_plain100x['discount_type'] != "일반"]
    df_etc['discount_name'] = df_etc['discount_type']
    df_etc['discount_rate'] = -1 

    # 3. %가 포함된 df_rate 처리
    # if price == 0 : df_rate에서도 가격이 0이라면 discount_rate는 100이므로, % 데이터를 무시하고 100으로 처리.
    # if price != 0 : "~%할인", "~%_"로 끝나는 데이터 처리 후, 할인 이름과 할인률을 추출.
    df_rate100 = df_rate[df_rate['price'] == 0]
    df_rate100['discount_name'] = df_rate100['discount_type']
    df_rate100['discount_rate'] = 100

    df_rate100x = df_rate[df_rate['price'] != 0]    
    df_rate100x['discount_type'] = df_rate100x['discount_type'].replace("_$","",regex=True) 
    df_rate100x['discount_type'] = df_rate100x['discount_type'].replace("할인$","",regex=True) 
    df_rate100x['discount_name'] = df_rate100x['discount_type'].apply(lambda x: re.sub(r'(\d+)%$', '', x))
    df_rate100x['discount_rate'] = df_rate100x['discount_type'].str.extract(r'(\d+)%$')

    # 4. df_new 생성, data type 통일
    df_new = pd.concat([df_plain100, df_plain100x, df_etc, df_rate100, df_rate100x])
    df_new['discount_rate'] = df_new['discount_rate'].astype(int)
    result = df_new.drop(['discount_type'], axis=1)
    return result.sort_index()

# 3. memeber_yn == 'N' 오류 처리  
member_yn = 'N' 인데 멤버십, 젠더 정보가 있는 경우가 있음  
멤버십, 젠더의 정보가 있다는 것은 회원이라는 것을 의미하므로 Y로 되어야 함

In [ ]:
def error_member_yn(df):
    idx = df.query('`member_yn` == "N" and `gender` == `gender`').index
    df.loc[idx,'member_yn'] = 'Y'
    
    return df

# 4. data labeling  
  
  * pre_open_date  
    값이 있다면(선예매 있는 공연) 1, 값이 없다면(선예매 없는 공연) 0

  * gender  
    여자는 1, 남자는 0, Nan은 -1로 Labeling

  * member_yn
    Y는 1로 N 은 0

  * ticket_cancel
    2 : 취소, 0 : 취소X -> 1: 취소, 0 : 취소X로 변경

In [ ]:
def labeling(df):
    
    # 1. pre_open_date 
    x = np.array(df.pre_open_date) # pre_open_date 값들을 numpy array배열로 변환 
    df.pre_open_date = np.where(np.isnan(x), 0, 1) # 값이 존재하면 1(선예매 O), 없으면 0(선예매 X) 으로 변환

    # 2. gender
    n = len(df)
    label = []
    for i in range(n):
        if df['gender'][i] == 'M':
            label.append(0)
        elif df['gender'][i] == 'F':
            label.append(1)
        else:
            label.append(-1)
            
        if df['ticket_cancel'][i] == 2:
            df['ticket_cancel'][i] == 1
            
    df.gender = label
            
    # 3. member_yn
    x = np.array(df.member_yn)
    df.member_yn = np.where(x == 'Y', 1, 0)

    return df

## 5. membership column 정리  

In [ ]:
 def membership_col_make(df):
    
    #data를 member, nonmember를 나눠줌
    member_df = df.query('`member_yn` == 1')
    nonmember_df = df.query('`member_yn` == 0')
    
    # nonmember free 및 paid 비회원 처리
    nonmember_df['membership_free'] = '비회원'
    nonmember_df['membership_paid'] = '비회원'
    
    # free, paid 타입 구분
    free = ['노블', '싹틔우미', '무료'] # 무료는 알고리즘 내에서 '일반'으로 처리함
    paid = ['법인', '골드', '블루', '그린'] # 다 해당하지 않으면 무료, index가 작을수록 높은등급
    
    membership_free = []
    membership_paid = []
    # 6번 type은 모두 nan이므로 제외 
    membership_list = member_df[['age', 'membership_type_1','membership_type_2','membership_type_3','membership_type_4','membership_type_5']].values
    
    # 일반 25 이상(직장인), 싹틔우미 ~24(대학생 이하), 노블 69~
    for i in range(len(membership_list)):
        tmp_free = []
        tmp_paid = []
        m_types = membership_list[i]
        
        # 5개의 멤버십 타입중, free에 해당하는 것들을 tmp_free에 넣고 나머지는 tmp_paid에 넣음
        for j in range(1, 6):
            if m_types[j] in ['노블', '무료', '싹틔우미']:
                tmp_free.append(m_types[j])
                
            else:
                tmp_paid.append(m_types[j])
                
        # paid의 nan 중복제거
        tmp_paid = set(tmp_paid)
        
        # '무료'만 있거나 노블, 싹틔우미가 동시에 있는 경우
        if len(tmp_free) == 1 or len(tmp_free) == 3:
            # 나이정보를 결합해 입력
            if m_types[0] < 20:
                membership_free.append('싹틔우미')
                
            elif m_types[0] == 20:
                if len(tmp_paid) > 1:
                    membership_free.append('일반')
                    
                else:
                    membership_free.append('싹틔우미')
                    
            elif m_types[0] < 70:
                membership_free.append('일반')
            
            # 만약 나이정보가 없다면 '처리'를 넣어 나중에 추가처리
            else:
                if m_types[0] == m_types[0]:
                    membership_free.append('노블')
                else:
                    membership_free.append('처리')
                    
        # 이상이 없는 경우 주어진 값대로 넣어줌
        else:
            if tmp_free[0] != '무료':
                membership_free.append(tmp_free[0])
            else:
                membership_free.append(tmp_free[1])
        
        
        if len(tmp_paid) == 1: # Nan만 있다면 => 순수 무료회원
            membership_paid.append('무료')
        
        # Nan이 없다면 queue구조를 활용해 높은 등급의 멤버십만 남겨 데이터에 추가
        else: 
            queue = list(tmp_paid)
            ranking = 4
            
            while len(queue) > 1:
                x = queue.pop(0)
                
                if x in paid:
                    r = paid.index(x)
                    if r < ranking:
                        ranking = r
                        queue.append(x)
            
            membership_paid.append(queue[0])

    member_df['membership_free'] = membership_free
    member_df['membership_paid'] = membership_paid
    
    #member, nonmember 병합
    result = pd.concat([member_df, nonmember_df], axis = 0)
    
    # 기존 membership_type column 삭제 
    del_col = ['membership_type_1', 'membership_type_2', 'membership_type_3', 'membership_type_4', 'membership_type_5', 'membership_type_6']
    result.drop(columns = del_col, inplace = True)
    
    return result.sort_index()

# 6. age 결측값 대치  
* age가 nan, membership_free 존재 => free 등급 기준 평균대치 (일반의 경우 50대로 대치)  
* age가 nan, membership_free 미존재 => 삭제

In [ ]:
def age_nan(df):
    fill_일반 = np.floor(df.groupby('membership_free')['age'].mean().일반/10)*10
    fill_새싹 = round(df.groupby('membership_free')['age'].mean().싹틔우미, -1)
    fill_노블 = round(df.groupby('membership_free')['age'].mean().노블, -1)
    
    # 결측 대치할 index 추출
    index_fill = df.query('`member_yn` == 1 and `membership_free` != "처리" and `age` != `age`').index
    index_del = df.query('`membership_free` == "처리"').index
    
    # 결측값 대치
    for idx in index_fill:
        member_type = df.loc[idx, 'membership_free']
        if member_type == '일반':
            df.loc[idx, 'age'] = fill_일반
        elif member_type == '새싹':
            df.loc[idx, 'age'] = fill_새싹
        else:
            df.loc[idx, 'age'] = fill_노블
            
    
    # 처리 데이터 삭제
    df.drop(index = index_del, inplace = True)
        
    df.age = df.age.fillna(-1)
    df.age = df.age.astype('int')
    
    return df.reset_index(drop = True)

## 7. genre 결측값 대치
- genre의 isnull 값은 '기타'로 대치

In [16]:
def genre_nan(df):
    df.genre = df.genre.fillna("기타")
    return df